<a href="https://colab.research.google.com/github/IanTirok/Moringa-DSP10-WEEK-2/blob/main/Moringa_Data_Science_Prep_W2_Independent_Project_2021_05_Ian_Tirok_SQL_Notebook%20IP%20FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. INITIALIZATION**

LOAD SQL AND CONECT TO DATABASE

In [1]:
# Loading our sql then connecting to the database.
%load_ext sql

%sql sqlite://

'Connected: @None'

IMPORT CSV AND PANDAS

In [3]:
# Importing csv to handle our csv files.
# Importing pandas in order to work with the dataframes in our csv files
#
import csv
import pandas as pd

UPLOADING GRAND ELECTORS CSV FILE

In [4]:
# Loading GrandElectors_by_state.csv 
# Persist our Electors table with new values.
#
with open('/content/GrandElectors_by_state.csv', 'r')as f:
  GrandElectors = pd.read_csv(f, index_col=0, encoding='utf-8')

#%sql DROP IF EXISTS GrandElectors;
%sql PERSIST GrandElectors;

%sql SELECT * FROM GrandElectors LIMIT 5

 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


UPLOADING POPULATION BY STATE CSV FILE

In [5]:
# Loading Population_by_state.csv 
# Dropping the table population_by_state if it exists in our database.
# Persist our population_by_state table with new values.
#
with open('/content/Population_by_state.csv', 'r')as f:
  population = pd.read_csv(f, index_col=0, encoding='utf-8')

%sql PERSIST population;

%sql SELECT * FROM population LIMIT 5

 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


**2. JOINING THE TWO TABLES**

PREVIEW OF SETTING GrandElectors.State to CAPS

In [25]:
#Due to the mismatch in formatting of caps, we will set state in Grand electors table to CAPS
%%sql
SELECT UPPER(State) 
FROM GrandElectors
LIMIT 3;



 * sqlite://
Done.


UPPER(State)
ALABAMA
ALASKA
ARIZONA


ALTERING VALUES OF GrandElectors.State to CAPS

In [16]:
%sql UPDATE GrandElectors set state = UPPER(State);

#All Rows were affected

 * sqlite://
51 rows affected.


ResourceClosedError: ignored

**3. TESTING DATA INTEGRITY** 

CONFIRM VALUE OF GrandElectors.State is now CAPS

In [17]:
%sql SELECT * FROM GrandElectors LIMIT 3;

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11


FINDING OUT NUMBER OF ENTRIES IN population TABLE

In [ ]:
#Confirm number of entries in population table

%%sql 
SELECT COUNT(state)
FROM population;

 * sqlite://
Done.


COUNT(state)
56


FINDING OUT NUMBER OF ENTRIES IN GrandElectors_table

In [ ]:
#Confirm number of entries in GrandElectors_table

%%sql 
SELECT COUNT(state)
FROM GrandElectors;

 * sqlite://
Done.


COUNT(state)
51


In [ ]:
%%sql SELECT COUNT(state)
FROM GrandElectors
WHERE EXISTS (SELECT * FROM GrandElectors, population
              WHERE (GrandElectors.state) == (population.State));

 * sqlite://
Done.


COUNT(state)
51


RECOMMENDATION OF DATA INTEGRITY TEST

In [ ]:
#There seems to be 5 extra entries on the population table
#There are 51 entries that appear on both tables
#By nerging the two tables and only selecting entries that match on both tables,
#The code below will allow us to work with only entries that appear on the state collumn on both tables


**4. MERGING THE TWO TABLES TO CREATE FullData_table**

THE MERGER

In [28]:
%%sql

create table FullData_table as 
SELECT GrandElectors.state, population, GrandElectors
FROM GrandElectors
INNER JOIN population
ON GrandElectors.state = population.State;


 * sqlite://
(sqlite3.OperationalError) table FullData_table already exists
[SQL: create table FullData_table as 
SELECT GrandElectors.state, population, GrandElectors
FROM GrandElectors
INNER JOIN population
ON GrandElectors.state = population.State;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


PREVIEW FullData_Table


In [29]:
%%sql
SELECT * FROM FullData_table
LIMIT 3
;

 * sqlite://
Done.


State,Population,GrandElectors,Ratio
ALABAMA,4858979,9,539886.0
ALASKA,738432,3,246144.0
ARIZONA,6828065,11,620733.0


**4. ALTER FULLDATA_table TO ADD RATIO COLUMN**

CREATING A COLUMN AND CALLING IT Ratio

In [10]:
%%sql
ALTER TABLE FullData_table ADD COLUMN Ratio float(10);
SELECT * FROM FullData_table
LIMIT 3;

 * sqlite://
Done.
Done.


State,Population,GrandElectors,Ratio
ALABAMA,4858979,9,None
ALASKA,738432,3,None
ARIZONA,6828065,11,None


CALCULATE AND SET RATIO TO RATIO COLUMN ON FullData_table


In [12]:
%%sql
UPDATE FullData_table SET Ratio = Population/GrandElectors;
SELECT * FROM FullData_table
LIMIT 3;

 * sqlite://
51 rows affected.
Done.


State,Population,GrandElectors,Ratio
ALABAMA,4858979,9,539886.0
ALASKA,738432,3,246144.0
ARIZONA,6828065,11,620733.0


**5. UPDATE FullData_table CHANGING DISTRICT OF COLUMBIA TO DC**

In [18]:
%%sql
UPDATE FullData_table
SET State = "DC" 
WHERE state = "DISTRICT OF COLUMBIA";

 * sqlite://
1 rows affected.


ResourceClosedError: ignored

**6. ORDER FullData_table BY Ratio PER CAPITA**

PRIORITY LIST OF GRAND ELECTORS PER CAPITA

In [31]:
%%sql
select *
FROM FullData_table 
ORDER BY Ratio DESC
LIMIT 12
;

 * sqlite://
Done.


State,Population,GrandElectors,Ratio
TEXAS,27469114,38,722871.0
CALIFORNIA,39144818,55,711723.0
FLORIDA,20271272,29,699009.0
NEW YORK,19795791,29,682613.0
NORTH CAROLINA,10042802,15,669520.0
OHIO,11613423,18,645190.0
VIRGINIA,8382993,13,644845.0
ILLINOIS,12859995,20,642999.0
PENNSYLVANIA,12802503,20,640125.0
NEW JERSEY,8958013,14,639858.0


**7.COMPUTE RUNNING TOTAL**

In [33]:
%%sql
SELECT a1.State, a1.GrandElectors, SUM(a2.GrandElectors) Running_Total
FROM FullData_table a1, FullData_table a2
WHERE a1.GrandElectors <= a2.GrandElectors or (a1.GrandElectors=a2.GrandElectors and a1.State = a2.State)
GROUP BY a1.State, a1.GrandElectors
ORDER BY a1.GrandElectors DESC, a1.State DESC
LIMIT 12
;

 * sqlite://
Done.


State,GrandElectors,Running_Total
CALIFORNIA,55,55
TEXAS,38,93
NEW YORK,29,151
FLORIDA,29,151
PENNSYLVANIA,20,191
ILLINOIS,20,191
OHIO,18,209
MICHIGAN,16,241
GEORGIA,16,241
NORTH CAROLINA,15,256


**8. CALCULATE GRAND ELECTORS THRESHOLD TO WIN THE ELECTION**

SUM OF TOTAL GRAND ELECTORS

In [21]:
%%sql
SELECT
SUM(GrandElectors)  
FROM
FullData_table
;

 * sqlite://
Done.


SUM(GrandElectors)
538


COMPUTE HALF OF THE TOTAL GRAND ELECTORS  

In [22]:
%%sql
SELECT
SUM(GrandElectors)/2  
FROM
FullData_table;

 * sqlite://
Done.


SUM(GrandElectors)/2
269


**9. IDENTIFY PRIORITY STATES TO REACH COMPUTED THRESHOLD (269)**

In [45]:
%%sql
SELECT a1.State, a1.GrandElectors, SUM(a2.GrandElectors) Running_Total
FROM FullData_table a1, FullData_table a2
WHERE a1.GrandElectors <= a2.GrandElectors or (a1.GrandElectors=a2.GrandElectors and a1.State = a2.State)

GROUP BY a1.State, a1.GrandElectors
ORDER BY a1.GrandElectors DESC, a1.State DESC

LIMIT 12

;

 * sqlite://
Done.


State,GrandElectors,Running_Total
CALIFORNIA,55,55
TEXAS,38,93
NEW YORK,29,151
FLORIDA,29,151
PENNSYLVANIA,20,191
ILLINOIS,20,191
OHIO,18,209
MICHIGAN,16,241
GEORGIA,16,241
NORTH CAROLINA,15,256
